<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/November_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install firebase-admin 

In [0]:
import firebase_admin
from firebase_admin import credentials, db, storage 
import cv2 
import math
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt 
import csv 
import pandas as pd


In [0]:
cred = credentials.Certificate("/content/celestinihdr-2-firebase-adminsdk-x7gew-beb69daef0.json")
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'celestinihdr-2.appspot.com',
}, name='novemeber')

In [14]:
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

anandvihar_hdr=[]
wazirpur_hdr=[]
wazirpur_jpeg={}
shadipur_jpeg={}
ctr=0
for blob in blobs:
  if(blob.name[7:13]=='static' and blob.name[14]=='P' and blob.name[27]=='j' and int(blob.name[43:45])<18 and int(blob.name[43:45])>6):
    try:
      blob.download_to_filename('/content/' + "wazirpur_jpeg" + str(blob.name.split('/')[-1])+ '.jpeg')
      name = "wazirpur_jpeg" + str(blob.name.split('/')[-1])+ '.jpeg' 
      ctr+=1
      print(ctr)
      im = cv2.imread(name)
      im = cv2.resize(im,(256,256))
      wazirpur_jpeg[blob.name]=im
    except:
      print(blob.name)
      
  if(blob.name[7:13]=='static' and blob.name[14]=='P' and blob.name[25]=='j'):
    try:
      blob.download_to_filename('/content/' + "shadipur_jpeg" + str(blob.name.split('/')[-1])+ '.jpeg')
      name = "shadipur_jpeg" + str(blob.name.split('/')[-1])+ '.jpeg' 
      ctr+=1
      print(ctr)
      im = cv2.imread(name)
      im = cv2.resize(im,(256,256))
      #Extra step of rotation 
      im = cv2.rotate(im,rotateCode =cv2.ROTATE_90_CLOCKWISE)
      shadipur_jpeg[blob.name]=im
    except:
      print(blob.name)
  
     
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
images/static/PatelNagar/jpeg/2019-11-18 10:08:06_Shadipur, New Delhi - CPCB.jpg
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257

In [16]:
print(len(shadipur_jpeg))
print(len(wazirpur_jpeg))

74
287


IMAGE PARAMETERS 

In [0]:
def trans_deg(im):
  B=np.zeros((256,256))
  D=np.zeros((256,256))
  C=np.zeros((256,256))
  # 3rd channel or im[2] has been split as b,g,r 

  b,g,r = cv2.split(im)

  for i in range(b.shape[0]):
    for j in range(b.shape[1]):
      D[i][j] = min(b[i][j], g[i][j], r[i][j])
      B[i][j] = max(b[i][j], g[i][j], r[i][j])
      C[i][j] = B[i][j] - D[i][j]

  d = np.sum(D)/float(np.size(D))
  b = np.sum(B)/float(np.size(B))
  c = b-d 
  A = (0.33)*max(np.ndarray.flatten(B)) + (0.66)*b
  
  x1=((A - d)/float(A)) 
  x2=(c/float(A))
  u = 10.127489
  v= -8.336512
  s = 0.13606234
  
  w = math.exp(-0.5*(u*x1 + v*x2)+s)
  
  if(w<=0.1):
      w=0
  elif(w<=0.3):
    w=1
  elif(w<=0.5):
    w=2
  elif(w<=0.7):
    w=3
  elif(w<=0.8):
    w=4
  elif(w<=0.9):
    w=5
    
  return w

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
    
  return (-1 * ent[0])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont

In [23]:
for i in wazirpur_jpeg:
  print(i[32:48])
  break

for i in shadipur_jpeg:
  print(i[30:46])
  break

2019-11-12 07:43
2019-11-18 07:18


In [26]:
feat=[]
ctr=0
for i in wazirpur_jpeg:
  l=[]
  l.append(i[32:48])
  l.append(entropy(wazirpur_jpeg[i]))
  l.append(contrast(wazirpur_jpeg[i]))
  l.append(trans_deg(wazirpur_jpeg[i]))
  feat.append(l)
  ctr+=1
  print(ctr)
  

for i in shadipur_jpeg:
  l=[]
  l.append(i[30:46]) ##
  l.append(entropy(shadipur_jpeg[i]))
  l.append(contrast(shadipur_jpeg[i]))
  l.append(trans_deg(shadipur_jpeg[i]))
  feat.append(l)
  ctr+=1
  print(ctr)
  



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [27]:
print(feat)

[['2019-11-12 07:43', 38.89207458496094, 74.3979487461643, 1], ['2019-11-12 07:58', 39.154083251953125, 74.91918491287663, 1], ['2019-11-12 08:03', 46.593894958496094, 67.04632812972218, 1], ['2019-11-12 08:14', 31.567014694213867, 48.1650281549805, 1], ['2019-11-12 08:15', 1.9462599754333496, 0.9573154917779065, 2], ['2019-11-12 08:16', 2.1617543697357178, 1.0411184717041266, 3], ['2019-11-12 08:17', 1.972675085067749, 0.9803854394707262, 3], ['2019-11-12 08:18', 1.9383140802383423, 0.9545441345822664, 3], ['2019-11-12 08:20', 2.021919012069702, 0.9831113584504381, 3], ['2019-11-12 08:22', 46.88556671142578, 66.66611799699457, 1], ['2019-11-12 08:24', 51.72476577758789, 66.0936636141551, 1], ['2019-11-12 08:27', 27.29054832458496, 53.960970701543836, 3], ['2019-11-12 08:28', 43.50270080566406, 52.37041048684994, 2], ['2019-11-12 08:55', 22.77055549621582, 43.07882162357483, 4], ['2019-11-12 08:55', 22.566495895385742, 44.9361855178027, 2], ['2019-11-12 08:56', 22.390079498291016, 45.0

In [0]:
#IMAGE NAME MODIFICATION - for label matching 

image_match=[]
for l in feat:
  temp = l[0]
  #print(temp)
  temp=temp[-24:-1]
  #print(temp)
  temp=temp.split('-')
  #print(temp)
  newtemp=temp[2].split(' ')
  #print(newtemp)
  newesttemp=newtemp[1].split(':')
  #print(newesttemp)
  j=int(newesttemp[1])
  if(j<=7.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  elif(j<=15 or j<=22.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '15'
  elif(j<=30 or j<=37.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '30'
  elif(j<=45 or j<=52.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '45'
  elif(j>52.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  t=[]
  t.append(final)
  #t.append(final[11:13])
  t.append(l[1])
  t.append(l[2])
  t.append(l[3])
  image_match.append(t)

  

In [29]:
print(len(image_match))

361


In [30]:
print(image_match)

[['12-11-2019 07:00', 38.89207458496094, 74.3979487461643, 1], ['12-11-2019 07:00', 39.154083251953125, 74.91918491287663, 1], ['12-11-2019 08:00', 46.593894958496094, 67.04632812972218, 1], ['12-11-2019 08:00', 31.567014694213867, 48.1650281549805, 1], ['12-11-2019 08:00', 1.9462599754333496, 0.9573154917779065, 2], ['12-11-2019 08:00', 2.1617543697357178, 1.0411184717041266, 3], ['12-11-2019 08:00', 1.972675085067749, 0.9803854394707262, 3], ['12-11-2019 08:00', 1.9383140802383423, 0.9545441345822664, 3], ['12-11-2019 08:00', 2.021919012069702, 0.9831113584504381, 3], ['12-11-2019 08:00', 46.88556671142578, 66.66611799699457, 1], ['12-11-2019 08:00', 51.72476577758789, 66.0936636141551, 1], ['12-11-2019 08:00', 27.29054832458496, 53.960970701543836, 3], ['12-11-2019 08:00', 43.50270080566406, 52.37041048684994, 2], ['12-11-2019 08:00', 22.77055549621582, 43.07882162357483, 4], ['12-11-2019 08:00', 22.566495895385742, 44.9361855178027, 2], ['12-11-2019 08:00', 22.390079498291016, 45.0

WEATHER PARAMETERS 

In [0]:
cred = credentials.Certificate('/content/credentials_weather.json')

# The weather data after 1st July 
# Initialize the app with a service account, granting admin privileges
new = firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://celestini-bd493.firebaseio.com/'
}, name ='w')

In [0]:

ref = db.reference(app=new)
df = ref.get()


In [0]:
temp_weather = {}

for key in df:
  if(key[0:10]=='12-11-2019'):
    l=[]
    l.append(df[key]['main']['humidity'])
    l.append(df[key]['main']['pressure'])
    l.append(df[key]['main']['temp'])
    l.append(df[key]['wind']['deg'])
    l.append(df[key]['wind']['speed'])
    temp_weather[key]=l
  if(key[0:10]=='18-11-2019'):
    l=[]
    l.append(df[key]['main']['humidity'])
    l.append(df[key]['main']['pressure'])
    l.append(df[key]['main']['temp'])
    l.append(df[key]['wind']['deg'])
    l.append(df[key]['wind']['speed'])
    temp_weather[key]=l
      

In [39]:
print(len(temp_weather))

534


In [0]:
for i in temp_weather:
  print(str(i) + '::' + str(temp_weather[i]) )


In [0]:
#WEATHER TMP MODIFICATION 

weather_match={}
for i in temp_weather:
  temp = i
  l=[]
  temp=temp[-24:-3]
  temp=temp.split('-')
  newtemp=temp[2].split(' ')
  newesttemp=newtemp[1].split(':')
  j=int(newesttemp[1])
  if(j<=7.5):
    final = temp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  elif(j<=15 or j<=22.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '15'
  elif(j<=30 or j<=37.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '30'
  elif(j<=45 or j<=52.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '45'
  elif(j>52.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '00'
  weather_match[final]=temp_weather[i]
  

In [41]:
print(len(weather_match))
for i in weather_match:
  print(str(i) + '::' + str(weather_match[i]) )

223
12-11-12 01:00::[55, 1013, 18, 340, 2.1]
12-11-2019 01:15::[55, 1013, 18, 340, 2.1]
12-11-2019 01:30::[55, 1013, 18, 337, 1.88]
12-11-2019 01:45::[55, 1013, 18, 337, 1.88]
12-11-2019 01:00::[59, 1013, 17, 337, 1.88]
12-11-12 02:00::[59, 1013, 17, 337, 1.88]
12-11-2019 02:15::[59, 1013, 17, 337, 1.88]
12-11-2019 02:30::[63, 1013, 17, 337, 1.88]
12-11-2019 02:45::[63, 1013, 16, 337, 1.88]
12-11-2019 02:00::[63, 1013, 16, 337, 1.88]
12-11-12 03:00::[63, 1013, 16, 337, 1.88]
12-11-2019 03:15::[63, 1013, 16, 337, 1.88]
12-11-2019 03:30::[63, 1013, 16, 337, 1.88]
12-11-2019 03:45::[63, 1013, 16, 337, 1.88]
12-11-2019 03:00::[63, 1013, 16, 337, 1.88]
12-11-12 04:00::[63, 1013, 16, 337, 1.88]
12-11-2019 04:15::[67, 1012, 16, 341, 1.42]
12-11-2019 04:30::[67, 1012, 16, 341, 1.42]
12-11-2019 04:45::[67, 1013, 16, 341, 1.42]
12-11-2019 04:00::[67, 1013, 16, 341, 1.42]
12-11-12 05:00::[67, 1013, 16, 341, 1.42]
12-11-2019 05:15::[67, 1013, 16, 341, 1.42]
12-11-2019 05:30::[67, 1013, 16, 341, 1.

LABELS 

In [0]:
##WAZIRPUR LABELS 
df_ = pd.read_csv('/content/november_wazirpur.csv')
data_ = df_.mask(df_.eq('None')).dropna()
file_ = data_.to_csv('/content/wazirpur_labels_new.csv')

label =[]
k=0
with open('/content/wazirpur_labels_new.csv') as file:
  reader = csv.reader(file, delimiter =',')
  prev=row[3]
  for row in reader:
    k+=1
    if(k%7==0):
        prev=row[3]
    if(k>3):
      l=[]
      l.append(row[1]) #DATE 
      l.append(row[3]) #PM
      l.append(prev)
      label.append(l) 

In [45]:
print(len(label))

257


In [0]:
##SHADIPUR LABELS 
df_ = pd.read_csv('/content/shadipur_novemeber.csv')
data_ = df_.mask(df_.eq('None')).dropna()
file_ = data_.to_csv('/content/shadipur_novemeber_new.csv')

k=0
with open('/content/shadipur_novemeber_new.csv') as file:
  reader = csv.reader(file, delimiter =',')
  for row in reader:
    k+=1
    if(k%7==0):
        prev=row[3]
    if(k>3):
      l=[]
      l.append(row[1]) #DATE 
      l.append(row[3]) #PM
      l.append(prev)
      label.append(l) 

In [47]:
print(len(label))

433


In [0]:
for i in label:
  print(i) 

In [53]:
ctr=0
image_label=[]
for i in image_match:
  for l in label:
    if(l[0]==i[0]):
      print(l[0])
      a=[]
      a.append(i[0])
      a.append(i[1])
      a.append(i[2])
      a.append(i[3])
      a.append(l[1])
      a.append(l[2])
      image_label.append(a)
      ctr+=1


12-11-2019 07:00
12-11-2019 07:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 08:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:00
12-11-2019 09:

In [52]:
print(len(image_label))
print(image_label[0])
print(image_label[120])


356
['12-11-2019 07:00', 38.89207458496094, 74.3979487461643, 1, '721', '721']
['12-11-2019 10:00', 11.674785614013672, 21.714368153723402, 0.9338180787990976, '621', '621']


In [0]:
#image, weather, label 
ctr=0
final=[]
for i in image_label:
  for w in weather_match:
    if(str(i[0])==str(w)):
      l=[]
      l.append(i[0][11:13])
      l.append(i[1])
      l.append(i[2])
      l.append(i[3])
      l.append(i[4])
      l.append(i[5])
      l.append(weather_match[w][0])
      l.append(weather_match[w][1])
      l.append(weather_match[w][2])
      l.append(weather_match[w][3])
      l.append(weather_match[w][4])
      final.append(l)
    
    


In [55]:
print(len(final))

356


In [58]:
print(final[355])

['10', 43.16773986816406, 59.40373829138724, 2, '76.36', '48.19', 28, 1015, 26.17, 303, 4.96]


In [0]:
trans=[]
cont=[]
ent=[]
hours=[]
hum=[]
pressure=[]
tempr=[]
deg=[]
speed=[]
prev=[]
pm=[]


for i in final:
  trans.append(float(i[3]))
  cont.append(float(i[2]))
  ent.append(float(i[1]))
  hours.append(float(i[0]))
  hum.append(float(i[6]))
  pressure.append(float(i[7]))
  tempr.append(float(i[8]))
  deg.append(float(i[9]))
  speed.append(float(i[10]))
  prev.append(float(i[5]))
  pm.append(float(i[4]))

In [60]:
print(len(trans))
print(len(hours))
print(len(cont))
print(len(ent))
print(len(hum))
print(len(pressure))
print(len(tempr))
print(len(deg))
print(len(speed))
print(len(pm))
print(len(prev))

356
356
356
356
356
356
356
356
356
356
356


In [61]:
print(trans)
print(hours)
print(cont)
print(ent)
print(hum)
print(pressure)
print(tempr)
print(deg)
print(speed)
print(pm)
print(prev)

[1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 3.0, 3.0, 1.0, 1.0, 3.0, 2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.2002123726289697, 1.200188330896185, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 1.2004426971102824, 1.1942853495509895, 1.2004209303861164, 1.20043555634912, 5.0, 5.0, 5.0, 0.9113704126710627, 0.9112542191129201, 4.0, 0.9080034710608167, 5.0, 4.0, 5.0, 5.0, 1.2013736863911495, 0.9062936777911292, 0.910806921430767, 5.0, 1.2003325762631987, 4.0, 5.0, 5.0, 1.201422007419827, 1.2013890087515808, 1.2013709545547493, 1.201374723253015, 0.9046597180375331, 5.0, 0.9106192631344567, 5.0, 5.0, 0.9153035400554819, 5.0, 0.9032325920756831, 0.9182985086664578, 5.0, 5.0, 1.201445058507884, 1.2014451396310417, 0.9078798711212646, 5.0, 0.9065617648631318, 1.2002777256076487, 5.0, 5.0, 0.9144800999847289, 1.2008185137126177, 1.2014580777075525, 1.2014724788056146, 1.2006836839191597, 5.0, 5.0, 5.0, 5.0, 1.2014315703473688, 1.2014367434758484, 0.9147

In [0]:
import pickle 
from sklearn.preprocessing import MinMaxScaler 

In [0]:
final_shape=len(hours)

#FEATURES 

trans= np.reshape(trans,(final_shape,1))
hours = np.reshape(hours,(final_shape,1))
cont = np.reshape(cont,(final_shape,1))
ent = np.reshape(ent,(final_shape,1))
hum = np.reshape(hum,(final_shape,1))
pressure = np.reshape(pressure,(final_shape,1))
tempr = np.reshape(tempr,(final_shape,1))
deg = np.reshape(deg,(final_shape,1))
speed = np.reshape(speed,(final_shape,1))
prev = np.reshape(prev,(final_shape,1))


features = np.hstack((hours, cont, ent , hum, tempr, deg, speed, pressure,trans,prev))

# print(max(hours))
# print(min(hours))


# print(max(cont))
# print(min(cont))


# print(max(ent))
# print(min(ent))


# print(max(hum))
# print(min(hum))


# print(max(tempr))
# print(min(tempr))


# print(max(deg))
# print(min(deg))


# print(max(speed))
# print(min(speed))


# print(max(pressure))
# print(min(pressure))


# print(max(trans))
# print(min(trans))

# print(max(prev))
# print(min(prev))

with open('/content/NOVEMBER_non_scaled_features.bin','wb') as file:
  pickle.dump(features, file)


pm = np.reshape(pm,(final_shape,1))


##SCALING 

s1 = MinMaxScaler()
hours = s1.fit_transform(hours)
s2 = MinMaxScaler()
cont = s2.fit_transform(cont)
s3 = MinMaxScaler()
ent = s3.fit_transform(ent)
s4 = MinMaxScaler()
hum = s4.fit_transform(hum)
s5 = MinMaxScaler()
pressure = s5.fit_transform(pressure)
s6 = MinMaxScaler()
tempr = s6.fit_transform(tempr)
s7 = MinMaxScaler()
deg = s7.fit_transform(deg)
s8 = MinMaxScaler()
speed = s8.fit_transform(speed)
s9 = MinMaxScaler()
prev = s9.fit_transform(prev)


with open('/content/S1.bin','wb') as file:
  pickle.dump(s1,file)
with open('/content/S2.bin','wb') as file:
  pickle.dump(s2,file)
with open('/content/S3.bin','wb') as file:
  pickle.dump(s3,file)
with open('/content/S4.bin','wb') as file:
  pickle.dump(s4,file)
with open('/content/S5.bin','wb') as file:
  pickle.dump(s5,file)
with open('/content/S6.bin','wb') as file:
  pickle.dump(s6,file)
with open('/content/S7.bin','wb') as file:
  pickle.dump(s7,file)
with open('/content/S8.bin','wb') as file:
  pickle.dump(s8,file)
 
features = np.hstack((hours, cont, ent , hum, tempr, deg, speed, pressure,trans,prev))



In [0]:
print(features)

[[0.         0.9929962  0.75184617 ... 0.33333333 1.         1.        ]
 [0.         1.         0.75691279 ... 0.33333333 1.         1.        ]
 [0.1        0.89421312 0.90078112 ... 0.66666667 1.         1.        ]
 ...
 [0.9        0.6243114  0.42114606 ... 0.         4.         0.        ]
 [0.9        0.62568304 0.42620236 ... 0.         4.         0.        ]
 [1.         0.68680941 0.40493424 ... 0.         4.         0.02212389]]


In [0]:
#print(trans)
#print(hours)
#print(cont)
#print(ent)
 print(hum)
# print(pressure)
# print(tempr)
# print(deg)
# print(speed)
# print(pm)
# print(prev)

In [0]:
with open('/content/features_November_w.bin','wb') as file:
  pickle.dump(features,file)
with open('/content/labels_November_w.bin','wb') as file:
  pickle.dump(pm,file)

FURTHER WORK

In [0]:
# if(blob.name[7:13]=='static' and blob.name[14]=='P' and blob.name[27]=='h' and int(blob.name[42:44])<18 and int(blob.name[42:44])>6): 
  #   blob.download_to_filename('/content/' + "wazirpur_hdr" + str(blob.name.split('/')[-1])+ '.jpeg')
  #   name = "wazirpur_hdr" + str(blob.name.split('/')[-1])+ '.jpeg' 
  #   ctr+=1
  #   print(ctr)
  #   im = cv2.imread(name)
  #   im = cv2.resize(im,(256,256))
  #   wazirpur_hdr.append(im)
  # if(blob.name[7:13]=='static' and blob.name[14]=='H' and blob.name[21]=='h'):
  #    blob.download_to_filename('/content/' + "anandvihar_hdr" + str(blob.name.split('/')[-1])+ '.jpeg')
  #    name = "anandvihar_hdr" + str(blob.name.split('/')[-1])+ '.jpeg' 
  #    ctr+=1
  #    print(ctr)
  #    im = cv2.imread(name)
  #    im = cv2.resize(im,(256,256))
  #    anandvihar_hdr.append(im)